# Visualizing tables as Charts

This notebook shows you how to plot columns of a table quickly as scatter charts or histograms in Firefly. You will also learn how to customize the charts using Plotly-style configuration and display spectra from supported type of table.

## Setup
First, we create a `FireflyClient` instance and open a Firefly viewer. See [Initializing a FireflyClient instance](./initializing-vanilla.html) for more details. Then, we display a catalog table from a URL so that we have table data to create charts. See [Displaying Tables and Catalogs](./viewing-tables.html) for more details.

In [ ]:
from firefly_client import FireflyClient

# Initialize a FireflyClient instance
fc = FireflyClient.make_client(url="https://irsa.ipac.caltech.edu/irsaviewer")

# Display a catalog table (from URL of 2MASS TAP query for M51)
table_url = "http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',202.4841667,47.23055556,0.125))=1"
table_id = '2mass-catalog'
fc.show_table(file_input=table_url, tbl_id=table_id, title='2MASS Catalog')

{'success': True}

.. note::
    Firefly automatically tries to plot the displayed table in the "Active Chart" tab (typically a ra-dec scatter plot for catalogs). But for full control over plotting, we need the following methods which will create new charts in the "Pinned Charts" tab. So make sure to switch to "Pinned Charts" tab to see the charts created in this notebook.

## Quick scatter plot
Use `show_xyplot` method of FireflyClient object for a straightforward scatter plot from table columns. X and Y axis column parameters (`xCol` and `yCol`) can be an expression involving one or more table columns. See API docs for more parameters to customize the scatter plot.

In [2]:
fc.show_xyplot(tbl_id=table_id, # specifies which table to plot from
               xCol='j_m', yCol='h_m-k_m')

{'success': True}

## Quick histogram
Use `show_histogram` method of FireflyClient object for a straightforward histogram from a table column. See API docs for more parameters to customize the histogram.

In [3]:
fc.show_histogram(tbl_id=table_id, col='j_m', numBins=30)

{'success': True}

## Configurable plots
Alternatively, more generic method `show_chart` can be used to create similar plot but with a lot more configuration parameters as [Plotly data/layout structure](https://plotly.com/javascript/reference/index/). This gives more control over marker style, axes' titles, etc. 

The `trace` dictionary in `data` array should have:
1. a Firefly-specific key `tbl_id` to specify which table to plot from
2. the `x` and `y` keys assigned to the column names or expressions prefixed with `tables::`.

In [4]:
trace0 = {
    # Firefly-specific key and values
    'tbl_id': table_id, 'x': "tables::j_m", 'y': "tables::h_m-k_m",
    # Plotly keys and values
    'type': 'scatter', 'mode': 'markers',
    'marker': dict(size=4, color='red', opacity=0.6)
    }
layout = {'title': '2MASS color-mag',
          'xaxis': dict(title='J'), 'yaxis': dict(title='H - K')} 

fc.show_chart(data=[trace0], layout=layout)

{'success': True}

Similarly, histograms can also be created using `show_chart` method but it has more Firefly-specific keys as shown below:

In [5]:
trace0 = dict(type='fireflyHistogram',
                marker=dict(color='rgba(153, 51, 153, 0.8)'),
                firefly=dict(
                    tbl_id=table_id,
                    options=dict(
                        algorithm='fixedsizeBins',
                        fixedBinSizeSelection='numBins',
                        numBins=30,
                        columnOrExpr='j_m'
                        )))

layout_hist = dict(title='Photometry histogram',
                    xaxis=dict(title='J (mag)'),
                    yaxis=dict(title='Number'))

fc.show_chart(layout=layout_hist, data=[trace0])

{'success': True}

## Spectra
If a table follows [IVOA Spectrum Data Model](https://www.ivoa.net/documents/SpectrumDM/), simply displaying the table will automatically create a spectrum plot in the "Active Chart" tab (make sure to switch to this tab). You can fine tune it directly in Firefly UI using the chart options dialog in the toolbar.

In [6]:
euclid_spec_url = 'http://irsa.ipac.caltech.edu/api/spectrumdm/convert/euclid/q1/SIR/102159776/EUC_SIR_W-COMBSPEC_102159776_2024-11-05T16:21:17.235160Z.fits?dataset_id=euclid_combspec&hdu=217'

fc.show_table(euclid_spec_url, title='Euclid Spectrum')

{'success': True}